In [1]:
import csv
import copy
import numpy as np
import random
from igraph import *
import math
import itertools
from itertools import groupby
from itertools import chain
from operator import itemgetter
from scipy.optimize import linear_sum_assignment
import time
import multiprocess
from networkx import Graph
from networkx import connected_components
from networkx.algorithms import bipartite
from networkx.algorithms.bipartite.matching import minimum_weight_full_matching, maximum_matching, hopcroft_karp_matching
from networkx.algorithms import maximal_matching
import random
import seaborn as sns
import pandas as pd

In [9]:
# 50 000 users - 10 favourites

big_likes = []
big_users = []
big_items = []

with open('bipartite_all.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        big_likes.append([row[0], row[1], 1])
        big_users.append(row[0])
        big_items.append(row[1])


big_users = np.unique(big_users).tolist()

big_items = np.unique(big_items).tolist()

big_user_items = {user: list(list(zip(*item))[1]) for user, item in groupby(big_likes, itemgetter(0))}

In [10]:
# k among n combinations
def combinations(args):
    user, items, k = args
    return np.array([[user, '&'.join(row)] for row in list(itertools.combinations(items, k))])


In [21]:

def full_match(users, items, user_items, likes):
    """
        Parameters
        ----------
            data: List[Tuple[int, int]]
                List of couples like (user_id, item_id).
                An item could be a song, an artist or something else.

        Returns
        ----------
            fingerprints: List[Tuple[int, List[int]]]
                List of couples like (user_id, fingerprint).
                Fingerprint being a list of items.
            cost: int
                Sum of the lengths of all of the fingerprints.
            k: int
                Length of the biggest fingerprint.
    """
    
    # Pour k allant de 1 à n?
    for k in range (1, 10):
        
        # Création des ensembles des items de taille allant de 1 à k
        if k == 1:
            G = Graph()
            # Add nodes with the node attribute "bipartite"
            G.add_nodes_from(users, bipartite=0)
            G.add_nodes_from(items, bipartite=1)
            # Add edges only between nodes of opposite node sets
            G.add_weighted_edges_from(likes)
            
        else:
            # Si on a un fingerprint pour chaque user - fin de la fonction
            if len(fingerprints) == len(users):
                return fingerprints
            
            # Sinon - générer les combinaisons qui existent parmi les ensebles d'items des users
            else:
                #print(len(np.unique(fingerprints.keys())), len(np.unique(users)))
                undefined_users = [u for u in users if u not in fingerprints.keys()]
                print(undefined_users)
                try:
                    # Create items combinations of size k
                    pool = multiprocess.Pool()
                    my_combinations = pool.map(combinations, 
                                               ([user, item_list, k] for user, item_list in user_items.items() 
                                                if len(item_list) >= k))
                    new_likes = []

                    
                    for i in range(0, len(my_combinations)):
                        for j in range(0, len(my_combinations[i])):
                            new_likes.append([my_combinations[i][j][0], my_combinations[i][j][1], k])
                            
                    new_items = np.array(np.unique(np.array(new_likes)[:,1]))
                    
                    G.add_nodes_from(new_items, bipartite=1) 
                    G.add_weighted_edges_from(new_likes)
                    
                # To make sure processes are closed in the end, even if errors happen
                finally: 
                    pool.close()
                    pool.join()

                    
        # Init fingerprints             
        fingerprints = {}
        # Go through connected components of the graph
        for c in connected_components(G):
            matchings = maximal_matching(G.subgraph(c))
            for m in matchings: 
                if m[0].lower().islower():
                    fingerprints[m[0]] = m[1]
                else:
                    fingerprints[m[1]] = m[0]
                    
        # Calculate sum_cards and avg_card 
        sum_cards = 0
        
        for f in fingerprints.values():
            sum_cards += f.count('&') + 1
        
        avg_card = sum_cards / len(fingerprints.values())
        #print('avg_card', avg_card)
        
        print('Nb fingerprints', len(fingerprints), 'Sum of fingerprints cardinals:', sum_cards, 'avg_card:', avg_card, 'k:', k)
        
        # If there is a fingerprint for each user - end of algorithm
        if len(fingerprints) == len(users):
            return fingerprints, sum_cards, avg_card, k
        
    return fingerprints, sum_cards, avg_card, k

In [22]:
# From 100 to 10 000 users

results = []

for power in range(2,5):
    
    avg_cards = []
    sum_cards = []
    ks = []
    
    for i in range(0,10):
        users = random.sample(big_users, k=10**power)
        print(len(np.unique(users))) 
        user_items = {}
        
        for u in users:
            if len(big_user_items[u]) <= 10:
                    user_items[u] = big_user_items[u]
            else:
                user_items[u] = random.sample(big_user_items[u], k=10)
            if len(user_items[u]) != len(np.unique(user_items[u])):
                print(len(user_items[u]), len(np.unique(user_items[u])))        
                
        items = np.unique(list(chain.from_iterable(list(user_items.values())))).tolist()

        likes = []
        for user, item_list in user_items.items():
            for item in item_list:
                likes.append([user, item, 1])

        # Fingerprints

        fingerprints, sum_card, avg_card, k = full_match(users, items, user_items, likes)
        
        avg_cards.append(avg_card)
        sum_cards.append(sum_card)
        ks.append(k)
    
    results.append({'nb_users': 10**power,
                    'avg_avg_cards': sum(avg_cards) / 10,
                    'avg_k': sum(ks) / 10,
                    'max_k': max(ks)})
    
    print({'nb_users': 10**power,
           'avg_avg_cards': sum(avg_cards) / 10,
           'avg_k': sum(ks) / 10,
           'max_k': max(ks)})

100
Nb fingerprints 100 Sum of fingerprints cardinals: 100 avg_card: 1.0 k: 1
100
Nb fingerprints 100 Sum of fingerprints cardinals: 100 avg_card: 1.0 k: 1
100
Nb fingerprints 100 Sum of fingerprints cardinals: 100 avg_card: 1.0 k: 1
100
Nb fingerprints 100 Sum of fingerprints cardinals: 100 avg_card: 1.0 k: 1
100
Nb fingerprints 98 Sum of fingerprints cardinals: 98 avg_card: 1.0 k: 1
['98b6bfb5bc3182c048ccdddf5ac973a7e3f85eb8', '34edd094447583eb630983ed65cb3b7ffa76106f']
Nb fingerprints 98 Sum of fingerprints cardinals: 105 avg_card: 1.0714285714285714 k: 2
['98b6bfb5bc3182c048ccdddf5ac973a7e3f85eb8', '34edd094447583eb630983ed65cb3b7ffa76106f']
Nb fingerprints 98 Sum of fingerprints cardinals: 118 avg_card: 1.2040816326530612 k: 3
['98b6bfb5bc3182c048ccdddf5ac973a7e3f85eb8', '34edd094447583eb630983ed65cb3b7ffa76106f']
Nb fingerprints 98 Sum of fingerprints cardinals: 123 avg_card: 1.2551020408163265 k: 4
['98b6bfb5bc3182c048ccdddf5ac973a7e3f85eb8', '34edd094447583eb630983ed65cb3b7ffa7

KeyboardInterrupt: 

It seems that taking 10 random items to define a user is not equivalent to taking users with exactly 10 favs.